# RecSys. Data

Идея - разработка сервиса (пока сайт, в светлом будущем и мобильное приложение :)), который будет рекомендовать книги, фильмы сериалы (а может быть еще статьи, например, на Хабре, подкасты, курсы и многое многое другое) в зависимости от того, сколько свободного времени есть у пользователя. Можно построить рекомендательную систему для нового контента и для контента, добавленного в "Избранное".

Вообще, в условиях нехватки данных для нового сервиса с новыми клиентами, есть желание построить модель, где, возможно, используются контекстуальные многорукие бандиты + классический RecSys. Из бонусов, данная иситема сможет подстраиваться на каждого уникального пользователя. Но пока воспользуемся для начала открытые датасеты о книгах, фильмах и сериалах (потом можно собрать датасет по статьям на Хабре и подкастам). А потом можно собрать маленькую тестовую группу, на которой проверить насколько хорошо работают алгоритмы такого обучения или настроить имитационную модель и проверять обучение на ней. 

Дальше можно собирать информацию о паттернах поведения пользователя, распозновать эмоции и все это учитывать при рекомендации

## Загрузка и предобработка данных

In [1]:
import pandas as pd
from pathlib import Path
from tqdm import tqdm

Данные: Так как идея в том, чтобы рекомендовать в зависимости от количества свободного времени, то нужно иметь, как возможность рекомендовать что-то длинное - фильм, что-то относительно короткое - сериалы, или что-то не сильно привязаное ко времени - книги (статьи)

Данные: 
Книги - Book-Crossing Dataset (http://www2.informatik.uni-freiburg.de/~cziegler/BX/)

Фильмы - Movielens (https://grouplens.org/datasets/movielens/)

Фильмы и сериалы Kion - RecSys Course Competition (https://ods.ai/competitions/competition-recsys-21/data)

Немного разношерстно (русский и английский языки), но для начала сойдет. 

In [2]:
books_ds_path = Path('datasets/books/')
films_ds_path = Path('datasets/films/')
kion_ds_path = Path('datasets/kion/')

Все данные, которые есть на данном этапе, представленны ниже. Но baseline и БД c клиентами пока что построим на основе рейтинг таблиц. БД с контентом возьмем готовые или вытянем из рейтинга

In [3]:
# Закомментрировал, чтобы не было проблем с памятью. Не удалил, потому что логично вливаются в рассказ и ход мыслей
# df_books_rating = pd.read_csv(books_ds_path / 'Book-Ratings.csv', encoding='ISO-8859-1', sep=';')
# df_books = pd.read_csv(books_ds_path / 'Books.csv', encoding='ISO-8859-1', sep=';')
# df_boooks_users = pd.read_csv(books_ds_path / 'Users.csv', encoding='ISO-8859-1', sep=';')

In [4]:
# df_films = pd.read_csv(films_ds_path / 'movies.csv')
# df_films_rating = pd.read_csv(films_ds_path / 'ratings.csv')

In [5]:
# df_kion = pd.read_csv(kion_ds_path / 'items.csv')
# df_kion_users = pd.read_csv(kion_ds_path / 'users.csv')
# df_kion_rating = pd.read_csv(kion_ds_path / 'interactions.csv')

In [97]:
df_books_rating

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149774,276704,1563526298,9
1149775,276706,0679447156,0
1149776,276709,0515107662,10
1149777,276721,0590442449,10


In [91]:
df_films_rating

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
25000090,162541,50872,4.5,1240953372
25000091,162541,55768,2.5,1240951998
25000092,162541,56176,2.0,1240950697
25000093,162541,58559,4.0,1240953434


In [98]:
df_kion_rating

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0
...,...,...,...,...,...
5476246,648596,12225,2021-08-13,76,0.0
5476247,546862,9673,2021-04-13,2308,49.0
5476248,697262,15297,2021-08-20,18307,63.0
5476249,384202,16197,2021-04-19,6203,100.0


План работы:

собрать общую БД по пользователям (простенькую)

Собрать 3 бд по контенту - фильмы, сериалы, книги 

Построить простенький рексус (избранное и/или новое)

Написать функцию для метрик, обосновать метрики и радоваться жизни

## Формулировка задачи

Нужна рекомендательная система, значит нужно выбрать из всего контента n штук (Пусть n = 20, будем рекомендовать контент в 4 страницы по 5 рекомендаций на странице). Успехом будет, если пользователь заинтересовался предложеным контентом, и чем он выше (на перовм месте на первой странице, лучше чем на первом месте, но на второй странице), тем лучше. Поэтому успехом будет интеракция (взаимодействие) пользователя с контентом. 

Будем считать, что паре клиент-контент (далее иногда user-item) ставится в оответствии 1, если интеракция была и 0, если интеракции не было. (Возможно имеет смысл ввести еще оценку -1, если интеракция была, но пользователю она не понравилась - он поставил мало баллов или бросил фильм/сериал на начале). Из пар user-item, у которых не было взаимодействия нужно выбрать такие, которые скорее всего заинтересуют пользователя. 

*(Еще одна мысль: Можно ставить 1, если взаимодейстие было, 0, если оно было не удачным и оставлять пустым, если интеракции не было, тогда можно предсказывать каким-то образом вероятность того, что пользователь откликнется. Скорее всего в будущем нужно будет попробовать разные алгоритмы рекомендаций и в некоторых такая вероятносная постановка будет работать лучше. Сейчас сказать сложно)*

Из таких размышлений, кажется, что логичнее всего выбрать метрику MAP@20. Она имеет логичное обоснование - чем выше метрика, тем чаще алгоритм выстраивает рекомендации так, что выше всего оказывается контент, который нравится пользователю. Так как пользователь может выбрать только один конкретный контент за раз, то можно домножить MAP@20 на 20 и получить еще более приятно трактуемую метрику, грубо трактуемую как степень близости выбронного клиентом контента к первому месту. (А если разделить 1 на получившуюся метрику, то также грубо можно трактовать ее как среднее место выбранного контента среди рекомендаций, и эту метрику надо уменьшать - 1/1 - в среднем на первом месте, 1/0.5 - в среднем на втором месте и так далее)

## Откладываем тестовую выборку

В будущем user-item матрицы, их разложения и многорукие бандиты, а пока предобработка и подготовка данных. Будем считать, что все эти пользоатели - пользователи нашего сервиса, просто кто-то выбирал только фильмы, кто-то только фильмы и сериалы, а кто-то только книги. Пока нет реального потока клиентов (хотя бы тестовых или имитированных) будем действовать по классике - train и test разбивка. Пока что будем считать, что спрос на контент пропорционален размерам датасетов, поэтому из каждого датасета возмем примерно по 20 процентов интеракций. 

In [3]:
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
RANDOM_SEED = 42

In [5]:
# Приведем датасеты с интеракциями к единому виду с едиными столбцоми.
# Пока не будем разделять на положительные и отрицательные интеракции

# Все id дополним буквами b, f и k, чтобы если вдруг есть одинаковые id они не перемешались. 
df_books_rating = pd.read_csv(books_ds_path / 'Book-Ratings.csv', encoding='ISO-8859-1', sep=';')
df_books_interactions = df_books_rating[['User-ID', 'ISBN']].rename({'User-ID': 'user', 'ISBN': 'item'}, axis=1)
df_books_interactions = 'b'+df_books_interactions.astype(str)
df_books_interactions['sourse'] = 'books'
del df_books_rating 

df_films_rating = pd.read_csv(films_ds_path / 'ratings.csv')
df_films_interactions = df_films_rating[['userId', 'movieId']].rename({'userId': 'user', 'movieId': 'item'}, axis=1)
df_films_interactions = 'f'+df_films_interactions.astype(str)
df_films_interactions['sourse'] = 'films'
del df_films_rating

df_kion_rating = pd.read_csv(kion_ds_path / 'interactions.csv')
df_kion_interactions = df_kion_rating[['user_id', 'item_id']].rename({'user_id': 'user', 'item_id': 'item'}, axis=1)
df_kion_interactions = 'k'+df_kion_interactions.astype(str)
df_kion_interactions['sourse'] = 'kion'
del df_kion_rating

In [6]:
train, test = [], []
for i in [df_books_interactions, df_films_interactions, df_kion_interactions]:
    df_train, df_test = train_test_split(i, test_size=0.2, random_state=RANDOM_SEED)
    del i
    train.append(df_train)
    test.append(df_test)
df_train = pd.concat(train)
df_test = pd.concat(test)

In [21]:
data_path = Path('data')

In [7]:
df_train.to_parquet(data_path / 'df_train.parquet')

In [ ]:
df_train.to_csv(data_path / 'df_train.csv')

In [8]:
df_test.to_parquet(data_path / 'df_test.parquet')

In [ ]:
df_test.to_parquet(data_path / 'df_test.csv')

### Соберем маленькую БД по Пользователям

Так как идея в том, чтобы рекомендовать что-то новое (или что-то из избранног), то нужно на новое и избранное разбить. Для этого заведем небольшую БД по пользователям, в котороый будут записаны их "Избранное" - то, что они уже смотрели/читали. Очевидно, что эта БД должна быть на основе train

In [17]:
users_bd = df_train.groupby(['user', 'sourse']).agg({'item': lambda x: ', '.join(x)})

In [22]:
users_bd.to_parquet(data_path / 'users_bd.parquet')